In [1]:
library("survey")

########################################################################################
### This takes the result of svyciprop and turns it into a simple vector
########################################################################################
tidyresult = function(res) {
    foo = c(mean(res),SE(res), confint(res))
    names(foo)  = NULL
    foo
}
    
#########################################################################################
### This puts a vector into string form.  I bet there is an R function that does this.
#########################################################################################
cc = function(v) {
    s = paste0("c(",v[1])
    v = v[-1]
    for (x in v) s = paste0(s,",",x)
    return(paste0(s,")"))
}

#########################################################################################
### This gets a dataframe containing just the year and state (or all US=XX) we are
### interested in.  There are three "big files", and we must first choose which file.
### We load a data frame from that file, then subset for state and year.
### The results of this function can be cached and used to call yrbssCalculate.
#########################################################################################
get.df.for.state.and.year = function(state, year.we.want, rda.path) {
    df = NULL
    if (state == 'XX') {
        load(paste0(rda.path,"sadc_2015_national.rda"))
        df = sadc.2015.national.df
    } else if (state <= 'MZ') {
        load(paste0(rda.path,"sadc_2015_state_a_m.rda"))
        df = sadc.2015.state.a.m.df
    } else {
        load(paste0(rda.path,"sadc_2015_state_n_z.rda"))
        df = sadc.2015.state.n.z.df
    }
    if (class(df) != 'data.frame') return("ERROR: Internal error: Missing/bad dataframe!")
    df = subset(df, df$sitecode == state)
    if (nrow(df)==0)  {
        return(paste0("ERROR: No data for state ",state,"!"))
    }
    #print(dim(df)) 
    df = subset(df, df$year == year.we.want)
    if (nrow(df)==0)  {
        return(paste0("ERROR: No data for state ",state," and year ",year.we.want,"!"))
    }
    #print(dim(df))
    return(df)
}
    


#########################################################################################
### This is called by yrbssCalculate.
### Given a filter criterion and a data frame, it computes a boolean vector that selects
### the desired rows from the dataframe.
#########################################################################################
construct.mask.from.filter = function(df, filter) {
    mask = TRUE
    for (var in names(filter)) {
        mask = mask & (df[[var]] %in% filter[[var]])
    }
    return(mask)
}

#########################################################################################
### This one verifies that the dataframe contains exactly state and year we are interested
### in.  It is used when yrbssCaculate is called with a cached dataframe.
#########################################################################################
verify.df = function(df, state, year.we.want) {
    states = unique(df$sitecode)
    if ((length(states) > 1) || states != state) {
        return(paste0("ERROR: df contains data for states other than ", state))
    }
    years = unique(df$year)
    if ((length(years) > 1) || years != year.we.want) {
        return(paste0("ERROR: df contains data for years other than ", year.we.want))
    }
    return(df)
}


#########################################################################################
### This is the main function.
#########################################################################################
yrbssCalculate =
  function (
    ### The default values are mostly just random examples.
    state = 'XX',   # XX=national, otherwise AL, AK, IN, NC, MT, etc.
    year = '2015',
    # directory where files are located, if not passing cached df:
    rda.path = "C:/Users/Rex/Desktop/OWH/owh-ds/software/owh/backoffice/yrbs/rda/",  
    # Dataframe for state and year, if cached. If missing, must pass rda.path:
    df = NULL,
    # Responses to be reported:
    positives = list(q8 = c(2, 4), q9 = c(1, 2)),
    # Filters (e.g. demographic) to be applied to respondents:
    filter = list(q1 = c(4, 5, 6), # age 15-18
                  q2 = a(1),   # female
                  q5 = a(3)),  # black
    group_by = list("sex","grade"),
    # List of variables that responses should be grouped by.
    # style of confidence interval; argumment to svyciprop.
    confint = "xlogit"  # logit, mean, beta, likelihood
  )
  {
    if (is.null(df)) df = get.df.for.state.and.year(state, year, rda.path)
    if (class(df) != 'data.frame') return(df)
    df = verify.df(df,state,year)
    if (class(df) != 'data.frame') return(df)  # it's an error message


    ### Create the "design" object for the weighted study design.
    design = svydesign(ids=~PSU, strata=~stratum, weights=~weight, data=df, nest=TRUE)

    ### Are all the column names mentioned in the arguments real?
    all.col.names = c(names(filter), names(positives), unlist(group_by))
    bad.col.names = all.col.names[!(all.col.names %in% colnames(df))]
    if (length(bad.col.names)>0) {
        return(paste0("ERROR: Bad column names: ", paste(bad.col.names,collapse=' '), '\n'))
    }
      
    ### From the filter criteria, construct a boolean vector to mask the rows.
    mask = construct.mask.from.filter(df, filter)
    #cat(sum(mask), "passed filters.\n")
    
    
    ### The group_by variables specify the dimensions of the final table.
    ### Get a list of possible values for each variable, then create a "grid" of
    ### all possible combinations of values.
    all_by_vals = lapply(group_by, function(s) c(sort(unique(df[[s]])),-1))
    names(all_by_vals) = group_by
    grid = expand.grid(all_by_vals)
    #print(grid)
    
    ### Create a row of column headings.
    kol.names =  c(unlist(group_by),
                     t(outer(names(positives),
                            c(".count", ".pct", ".se", ".ci.lo", ".ci.hi"),
                            paste0)))
    result = NULL
    result.string = paste0(paste0(kol.names, collapse=','),'\n')
    
    #print(result)        
    assigned = 0  # For grins, keep track of total number assigned to grid.
    
    ### for each cell in grid (i.e., each combination of group-by values)
    for (i in 1:nrow(grid)) {
        new.row = unlist(grid[i,])
        #print(new.row)
        ### Refine the mask to include only rows for this cell.
        fine.mask = mask
        #print(table(fine.mask, useNA="always"))
        for (s in group_by) {
            if (grid[i,s] == -1) next
            fine.mask = fine.mask & (!is.na(df[[s]]) & (df[[s]] == grid[i,s]))
            #print(table(fine.mask, useNA="always"))
        }
        #cat(sum(fine.mask), "filtered and in grid entry.\n")
        assigned = assigned + sum(fine.mask)
        
        ### for each question in positives, compute statistics for this grid cell.
        for (resp.name in names(positives)) {
            #cat("\nResponse",resp.name,"\n")
            resp.true = df[[resp.name]] %in% positives[[resp.name]]

            #cat(sum(resp.true), "interesting responses.\n")
            #cat(sum(resp.true & fine.mask), "interesting responses in this grid cell.\n")
            #cat(sum(fine.mask), "subjects in this grid cell.\n")

            count = sum(!is.na(df[[resp.name]]) & fine.mask)
            #cat("Count", count, "\n")
            
            if (count == 0) {
                new.row = c(new.row,count,0,0,0,0)
                next
            }
            proportion = sum(resp.true & fine.mask) / sum(fine.mask)
            #cat("Raw proportion", proportion, "\n")
            
            ss = subset(design,fine.mask)
            r.code = paste0("svyciprop(~(",resp.name,"%in%",cc(positives[[resp.name]]),"),",
                        "ss, method='xlogit', na.rm=TRUE)")
            #print(r.code)
            new.row = c(new.row,count,round(100*tidyresult(eval(parse(text=r.code))),2))

        }
        result = rbind(result,new.row)
        result.string = paste0(result.string,paste0(new.row, collapse=','),'\n')
    }
    #cat(assigned, "assigned to grid.\n")
    #cat("\n") 
    rownames(result) = NULL
    colnames(result) = kol.names
    #print(result) 
    #write.csv(result,"C:/Users/Rex/Desktop/foo.csv", row.names=FALSE)
    return(result.string)
  }
        
        
"OK"


Warning message:
"package 'survey' was built under R version 3.3.2"Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: 'survey'

The following object is masked from 'package:graphics':

    dotchart



[1] "OK"

In [2]:
###################################################
# Some test cases -- just a smoke test.
###################################################


rda.path = "C:/Users/Rex/Desktop/OWH/owh-ds/software/owh/backoffice/yrbs/rda/"
load(paste0(rda.path,"sadc_2015_national.rda"))
kolumns = colnames(sadc.2015.national.df)
#cat(kolumns)
odd = c(1,3,5,7,9)
gc()

### These just check that we can load the appropriate state and year, and return
### an error message if not.
dim.l.y = function(state,year) nrow(get.df.for.state.and.year(state,year,rda.path))    
print(dim.l.y('XX', 2015))
print(dim.l.y('XX', 2011))
print(dim.l.y('XX', 2009))
print(dim.l.y('MT', 2015))
print(dim.l.y('MT', 2011))
print(dim.l.y('MT', 2009))
print(dim.l.y('NC', 2015))
print(dim.l.y('NC', 2011))
print(dim.l.y('NC', 2009))
print(get.df.for.state.and.year('MT', 1909, rda.path))
gc()
    
      
### These are just a smoke test of yrbssCalculate.  Arguments that would normally be passed are
### allowed to default.
cat('\n\n', yrbssCalculate(filter=list(q8=1:2)) )
cat('\n\n', yrbssCalculate(filter=list(q8=1:2), state='MT', year = 2011) )
cat('\n\n', yrbssCalculate(filter=list(q8=1:2), state='MM', year = 2011) )
cat('\n\n', yrbssCalculate(filter=list(q9=c(1,3,5))) )
cat('\n\n', yrbssCalculate(filter=list(q8=1:2, q9=c(1,3,5))) )
cat('\n\n', yrbssCalculate(filter=list(q1=c(4, 5, 6), # age 15-18
                            q2=c(1), # female
                            q5=c("C","E")))) # bogus
cat('\n\n', yrbssCalculate(filter=list(queue8=1:2)) )  
    
### These examples illustrate caching.  We cache a dataframe for Montana/2011
### and use it twice.  Third time we get an error.
df.MT.2011 = get.df.for.state.and.year('MT',2011,rda.path)
cat('\n\n', yrbssCalculate(filter=list(q8=1:2), state='MT', year = 2011, df=df.MT.2011) )
cat('\n\n', yrbssCalculate(filter=list(q9=odd), state='MT', year = 2011, df=df.MT.2011) )
cat('\n\n', yrbssCalculate(filter=list(q8=1:2), state='MS', year = 2013, df=df.MT.2011) )


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1220501,65.2,2164898,115.7,1533655,82
Vcells,45048532,343.7,60626045,462.6,45080339,344


[1] 15624
[1] 15425
[1] 16410
[1] 4486
[1] 4148
[1] 1852
[1] 6178
[1] 2278
[1] 5702
[1] "ERROR: No data for state MT and year 1909!"
[1] "ERROR: No data for state NX!"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1225042,65.5,2164898,115.7,1533655,82.0
Vcells,45053670,343.8,176211294,1344.4,220249581,1680.4




 sex,grade,q8.count,q8.pct,q8.se,q8.ci.lo,q8.ci.hi,q9.count,q9.pct,q9.se,q9.ci.lo,q9.ci.hi
1,1,1327,58.26,2.72,52.68,63.63,1327,6.55,1.04,4.73,9.01
2,1,1281,74.06,2,69.81,77.9,1281,7.87,1.08,5.94,10.36
-1,1,2614,66.46,2.13,62.01,70.63,2614,7.25,0.82,5.75,9.11
1,2,1371,56.23,3.33,49.43,62.82,1371,4.81,0.97,3.19,7.19
2,2,1340,69.75,2.02,65.51,73.68,1340,8.55,1.44,6.04,11.96
-1,2,2725,62.72,1.91,58.78,66.5,2725,6.58,1.07,4.71,9.12
1,3,1454,48.89,3.51,41.86,55.97,1454,4.58,0.8,3.22,6.49
2,3,1427,66.36,2.43,61.27,71.1,1427,7.99,1.66,5.22,12.06
-1,3,2890,57.86,2.32,53.1,62.47,2890,6.52,1.14,4.56,9.24
1,4,1368,50.34,2.9,44.48,56.18,1368,5.32,0.96,3.68,7.63
2,4,1359,59.49,2.41,54.54,64.26,1359,6.58,1.35,4.32,9.89
-1,4,2730,54.91,2.05,50.73,59.02,2730,5.95,0.97,4.26,8.25
1,-1,5539,53.55,2.65,48.17,58.86,5539,5.39,0.7,4.14,7
2,-1,5435,67.54,1.33,64.79,70.18,5435,7.92,0.95,6.2,10.07
-1,-1,11018,60.59,1.69,57.12,63.96,11018,6.71,0.75,5.34,8.41


 sex,grade,q8.count,q8.pct,q8.se,q8.ci.lo,q8.ci.hi